In [3]:
# Load Our Dataset
import pandas as pd
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import qalsadi.lemmatizer
import time

In [5]:
#  data cleaning 
data=pd.read_excel('ar spam.xlsx')
data.head() 


,sent,txt
0,NaN,NaN
1,neg,#الاهلي_الهلال -تحت هذي التغريده تحدي روقان وا...
2,NaN,NaN
3,pos,صباح الخيرات يارب العالمين والصلاة على محمد وا...
4,NaN,NaN


In [6]:
#Removing Empty fields
data = data.dropna()

In [7]:
data.head()

,sent,txt
1,neg,#الاهلي_الهلال -تحت هذي التغريده تحدي روقان وا...
3,pos,صباح الخيرات يارب العالمين والصلاة على محمد وا...
5,mix,😂😂 فكري بعد ما بغير اسمي 😕 هذا عمر يجيب هالبلا...
7,neg,ذكرتك يوم أمطرت دارك وأنا بعيد وهلت دموعي على ...
9,neg,بس باقي انتظر الاذان وانام 😭


In [8]:
#Removing the mix rows
data = data[data["sent"].str.contains("mix") == False]
data.head()

,sent,txt
1,neg,#الاهلي_الهلال -تحت هذي التغريده تحدي روقان وا...
3,pos,صباح الخيرات يارب العالمين والصلاة على محمد وا...
7,neg,ذكرتك يوم أمطرت دارك وأنا بعيد وهلت دموعي على ...
9,neg,بس باقي انتظر الاذان وانام 😭
11,neg,تلاتين سنة بترقص .. الليلة رقصتنا أنا ببكي 😭 د...


In [10]:
# method to remove emoji's

def remove_emoji(text):
    non_arabic_char = re.compile('[^\s\\u0600-\u06FF]')
    text_with_no_spaces = re.sub(non_arabic_char, "", text)
    text_with_single_spaces = " ".join(re.split("\s+", text_with_no_spaces))
    
    return text_with_single_spaces

In [11]:
data["txt"]=data["txt"].map(remove_emoji) #map each row with remove_emoji's function
data.head()

,sent,txt
1,neg,الاهليالهلال تحت هذي التغريده تحدي روقان واسرع...
3,pos,صباح الخيرات يارب العالمين والصلاة على محمد وا...
7,neg,ذكرتك يوم أمطرت دارك وأنا بعيد وهلت دموعي على ...
9,neg,بس باقي انتظر الاذان وانام
11,neg,تلاتين سنة بترقص الليلة رقصتنا أنا ببكي دي حلا...


In [18]:
def tokenize_text(inp):
    return nltk.tokenize.wordpunct_tokenize(inp)
data_txt = data.txt.apply(lambda sentence: nltk.tokenize.wordpunct_tokenize(sentence))

In [19]:
data.txt = data.txt.apply(lambda sentence: nltk.tokenize.wordpunct_tokenize(sentence))
data.head()


,sent,txt
1,neg,"[الاهليالهلال, تحت, هذي, التغريده, تحدي, روقان..."
3,pos,"[صباح, الخيرات, يارب, العالمين, والصلاة, على, ..."
7,neg,"[ذكرتك, يوم, أمطرت, دارك, وأنا, بعيد, وهلت, دم..."
9,neg,"[بس, باقي, انتظر, الاذان, وانام]"
11,neg,"[تلاتين, سنة, بترقص, الليلة, رقصتنا, أنا, ببكي..."


In [20]:
def stopword_removal(inp):
    arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
    for i in inp:
        if i in arb_stopwords:
            inp.remove(i)

    return inp

In [21]:
data.txt=data["txt"].map(stopword_removal)

In [22]:
data.head()


,sent,txt
1,neg,"[الاهليالهلال, هذي, التغريده, تحدي, روقان, واس..."
3,pos,"[الخيرات, يارب, العالمين, والصلاة, محمد, وال, ..."
7,neg,"[ذكرتك, يوم, أمطرت, دارك, وأنا, بعيد, وهلت, دم..."
9,neg,"[باقي, انتظر, الاذان, وانام]"
11,neg,"[تلاتين, سنة, بترقص, الليلة, رقصتنا, ببكي, دي,..."


In [23]:
stemmer=nltk.ISRIStemmer()


In [25]:
def stem(text):
    out=[]
    for i in text:
        out.append(stemmer.stem(i))
            
    return out

In [26]:
start_time=time.time()
#data.txt.map(stem)

data.txt=data.txt.map(stem)
print("-------- ",(time.time() - start_time),' Secounds --------')

--------  3.625319480895996  Secounds --------


In [27]:
data.head()

,sent,txt
1,neg,"[اهليالهلال, هذي, غرد, تحد, روق, سرع, اذا, فاز..."
3,pos,"[خير, يرب, علم, صلة, حمد, وال, حمد]"
7,neg,"[ذكر, يوم, مطر, درك, وأن, بعد, وهل, دمع, لخد, ..."
9,neg,"[بقي, نظر, اذن, انم]"
11,neg,"[تلا, سنة, رقص, ليل, رقص, ببك, دي, حلو, شدد, باخ]"


In [28]:
import qalsadi.lemmatizer
lemmatizer = qalsadi.lemmatizer.Lemmatizer()

In [29]:
def lemmatize(text):
    out=[]
    for i in text:
        out.append(lemmatizer.lemmatize(i))

    return out

In [30]:
start_time=time.time()

#data.txt.map(lemmatize).head
#data.txt=data.txt.map(lemmatize)

print("-------- ",(time.time() - start_time),' Secounds --------')

--------  0.0  Secounds --------


In [22]:
# convert class labels to  Bad and  Good values

def decoder(arr):
    out=list()
    binary_list=list(arr)
    for item in binary_list:
        if item == 0:
            out.append('bad')
        else:
            out.append('good')
    return out

In [32]:
def join_txt(txt):
    
    return " ".join(txt)

In [33]:
data.txt=data.txt.map(join_txt)
data.head()

,sent,txt
1,neg,اهليالهلال هذي غرد تحد روق سرع اذا فاز هلل ان ...
3,pos,خير يرب علم صلة حمد وال حمد
7,neg,ذكر يوم مطر درك وأن بعد وهل دمع لخد عبر قلم جد...
9,neg,بقي نظر اذن انم
11,neg,تلا سنة رقص ليل رقص ببك دي حلو شدد باخ


In [38]:
# Feature extaction using Counter
bag_of_words_vectorizer=CountVectorizer() 
bag_of_words_count = bag_of_words_vectorizer.fit_transform(data["txt"])

In [39]:
from sklearn.model_selection import train_test_split
x_train_count, x_test_count, y_train_count, y_test_count = train_test_split(bag_of_words_count, data['sent'], random_state=42, test_size=0.25)

In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score

In [41]:
model_count=MultinomialNB()

In [42]:
model_count.fit(x_train_count,y_train_count)
pred = model_count.predict(x_test_count)
pred

array(['pos', 'pos', 'neg', ..., 'pos', 'pos', 'neg'], dtype='<U3')

In [43]:
accuracy = accuracy_score(y_test_count,pred) 
print("Model  had Trained and it's Accuracy: ", accuracy )

Model  had Trained and it's Accuracy:  0.7473684210526316


In [31]:
print(classification_report(y_test_count, pred))


              precision    recall  f1-score   support

         neg       0.73      0.75      0.74      6893
         pos       0.76      0.75      0.75      7452

    accuracy                           0.75     14345
   macro avg       0.75      0.75      0.75     14345
weighted avg       0.75      0.75      0.75     14345



In [32]:
print(confusion_matrix(y_test_count, pred))

[[5169 1724]
 [1900 5552]]


In [44]:
print(classification_report(y_test_count, pred))

pd.DataFrame(
    confusion_matrix(y_test_count, pred),
    index = [['actual', 'actual'], ['bad', 'good']],
    columns = [['predicted', 'predicted'], ['bad', 'good']])

              precision    recall  f1-score   support

         neg       0.73      0.75      0.74      6893
         pos       0.76      0.75      0.75      7452

    accuracy                           0.75     14345
   macro avg       0.75      0.75      0.75     14345
weighted avg       0.75      0.75      0.75     14345



predicted      
                  bad  good
actual bad       5169  1724
       good      1900  5552

In [45]:
#Binary Encoding
bag_of_words_vectorizer_binary=CountVectorizer(binary=True) 
bag_of_words_binary = bag_of_words_vectorizer_binary.fit_transform(data["txt"])

In [46]:
# data set between train and test 0.25 : 0.75
x_train_bin, x_test_bin, y_train_bin, y_test_bin = train_test_split(bag_of_words_binary, data['sent'], random_state=42, test_size=0.25)


In [47]:
# object from Naiv Base
model_bin=MultinomialNB()

In [48]:
# train model 
model_bin.fit(x_train_bin,y_train_bin)
# test  model
pred_bin = model_bin.predict(x_test_bin)
pred_bin

array(['pos', 'pos', 'neg', ..., 'pos', 'pos', 'neg'], dtype='<U3')

In [49]:
# accuracy precison
accuracy = accuracy_score(y_test_bin,pred_bin) 
print("Model  had Trained and it's Accuracy: ", accuracy )


Model  had Trained and it's Accuracy:  0.744719414430115


In [50]:
# our report about model after train and test 
print(classification_report(y_test_bin, pred_bin))


              precision    recall  f1-score   support

         neg       0.73      0.75      0.74      6893
         pos       0.76      0.74      0.75      7452

    accuracy                           0.74     14345
   macro avg       0.74      0.74      0.74     14345
weighted avg       0.75      0.74      0.74     14345



In [51]:
# confusion_matrix
print(confusion_matrix(y_test_bin, pred_bin))

[[5156 1737]
 [1925 5527]]


In [52]:
# true fasle
print(classification_report(y_test_bin, pred_bin))

pd.DataFrame(
    confusion_matrix(y_test_bin, pred_bin),
    index = [['actual', 'actual'], ['bad', 'good']],
    columns = [['predicted', 'predicted'], ['bad', 'good']])

              precision    recall  f1-score   support

         neg       0.73      0.75      0.74      6893
         pos       0.76      0.74      0.75      7452

    accuracy                           0.74     14345
   macro avg       0.74      0.74      0.74     14345
weighted avg       0.75      0.74      0.74     14345



predicted      
                  bad  good
actual bad       5156  1737
       good      1925  5527

In [53]:
#Feature extraction using TF-IDF

vectorizer = TfidfVectorizer()
bag_of_words_tfidf=vectorizer.fit_transform(data["txt"])

In [54]:
# data set between train and test 0.25 : 0.75

x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(bag_of_words_tfidf, data['sent'], random_state=42, test_size=0.25)


In [55]:
# object from Naiv Base
model_tfidf=MultinomialNB()

In [56]:
# train model 
model_tfidf.fit(x_train_tfidf,y_train_tfidf)
# test  model
pred_tfidf = model_tfidf.predict(x_test_tfidf)
pred_tfidf

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'neg'], dtype='<U3')

In [57]:
# accuracy precison
accuracy = accuracy_score(y_test_tfidf,pred_tfidf) 
print("Model  had Trained and it's Accuracy: ", accuracy )

Model  had Trained and it's Accuracy:  0.7499477169745556


In [58]:
# our report about model after train and test 
print(classification_report(y_test_tfidf, pred_tfidf))

              precision    recall  f1-score   support

         neg       0.74      0.74      0.74      6893
         pos       0.76      0.76      0.76      7452

    accuracy                           0.75     14345
   macro avg       0.75      0.75      0.75     14345
weighted avg       0.75      0.75      0.75     14345



In [59]:
# confusion_matrix
print(confusion_matrix(y_test_tfidf, pred_tfidf))

[[5075 1818]
 [1769 5683]]


In [60]:
# true fasle
print(classification_report(y_test_bin, pred_bin))

pd.DataFrame(
    confusion_matrix(y_test_bin, pred_bin),
    index = [['actual', 'actual'], ['bad', 'good']],
    columns = [['predicted', 'predicted'], ['bad', 'good']])

              precision    recall  f1-score   support

         neg       0.73      0.75      0.74      6893
         pos       0.76      0.74      0.75      7452

    accuracy                           0.74     14345
   macro avg       0.74      0.74      0.74     14345
weighted avg       0.75      0.74      0.74     14345



predicted      
                  bad  good
actual bad       5156  1737
       good      1925  5527